#Logistic Regression
#Creating a model if future applicant can get loan approved

In [2]:
#Importing libraries
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [3]:
#Loading the loan data
loan_df = pd.read_csv("loan_data.csv")
loan_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,1,NO,5849,0.0,40,360,1,Urban,1
1,LP001003,Male,Yes,1,1,NO,4583,1508.0,128,360,1,Rural,0
2,LP001005,Male,Yes,0,1,YES,3000,0.0,66,360,1,Urban,1
3,LP001006,Male,Yes,0,0,NO,2583,2358.0,120,360,1,Urban,1
4,LP001008,Male,No,0,1,NO,6000,0.0,141,360,1,Urban,1


In [11]:
#selecting some feature needed for the model
loan_df = loan_df[['Dependents', 'Education', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount','Loan_Amount_Term', 'Credit_History','Loan_Status']]
loan_df.head()

,Dependents,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,0,1,5849,0.0,40,360,1,1
1,1,1,4583,1508.0,128,360,1,0
2,0,1,3000,0.0,66,360,1,1
3,0,0,2583,2358.0,120,360,1,1
4,0,1,6000,0.0,141,360,1,1


In [12]:
#defining input X and target Y
X = np.asarray(loan_df[['Dependents', 'Education', 'ApplicantIncome', 'CoapplicantIncome','LoanAmount','Loan_Amount_Term', 'Credit_History']])
y = np.asarray(loan_df['Loan_Status'])
y [0:5]

array([1, 0, 1, 1, 1])

In [13]:
#normalizing the data set
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-0.74284285,  0.52261626,  0.08857424, -0.54251966, -1.2173378 ,
         0.28793353,  0.45738935],
       [ 0.25105337,  0.52261626, -0.12639978, -0.03796658, -0.17240278,
         0.28793353,  0.45738935],
       [-0.74284285,  0.52261626, -0.3952022 , -0.54251966, -0.908607  ,
         0.28793353,  0.45738935],
       [-0.74284285, -1.91344985, -0.46601117,  0.24643004, -0.26739687,
         0.28793353,  0.45738935],
       [-0.74284285,  0.52261626,  0.1142149 , -0.54251966, -0.01803737,
         0.28793353,  0.45738935]])

In [14]:
#splitting the data set into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (462, 7) (462,)
Test set: (116, 7) (116,)


In [16]:
#fitting model with training data set
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)


In [18]:
#predicting the data set
yhat = LR.predict(X_test)


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/base.py:283: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [20]:
yhat_prob = LR.predict_proba(X_test)


In [22]:
#using Jaccard index for accuracy evaluation
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat,pos_label=0)

ImportError: cannot import name 'jaccard_score' from 'sklearn.metrics' (/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/metrics/__init__.py)

In [23]:
#Using F1 score for accuracy
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

[[70  4]
 [27 15]]
